# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Признаки</p>
<p> f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);</p>
<p>product — объём запасов в скважине (тыс. баррелей)</p>


## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from sklearn.linear_model import LinearRegression
from scipy import stats as st

def data_read_glance(file_path):# создаем функцию для загрузки данных и первичного анализа. На вход принимает путь до файла.
    data = pd.read_csv(file_path)
    print(data.head(10))
    print(data.info())
    print('Данные о наличии дубликатов', data.duplicated().value_counts(),'\n')
    return data
reg0 = data_read_glance('C:/Users/VovAz/Desktop/Проект/Машинное обучение в бизнесе/geo_data_0.csv')
reg1 = data_read_glance('C:/Users/VovAz/Desktop/Проект/Машинное обучение в бизнесе/geo_data_1.csv')
reg2 = data_read_glance('C:/Users/VovAz/Desktop/Проект/Машинное обучение в бизнесе/geo_data_2.csv')

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Данные прочитаны, дубликатов нет, пропущенных значений нет. Типы данных не нуждаются в изменении.

## Обучение и проверка модели

In [2]:
def model_fit_valid(data):# создаем функцию для обучения модели, которая принимает датафрейм, печатает средний запас сырья,
    #согласно предсказаниям и RMSE модели, возваращает целевые признаки валидацинной выборки и предсказания
    features = data.drop(['id', 'product'], axis = 1)#выделяем параметры
    target = data['product']# выделяем целевые признаки
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)

    predict_mean = predict.mean()
    rmse = mean_squared_error(target_valid, predict)**0.5
   
    print('Cредний запас нефти: {:.5f}'.format(predict_mean))
    print('RMSE модели: {:.5f}'.format(rmse),'\n')
    
    return target_valid, predict

In [3]:
print("Показатели для региона 0:")
reg0_valid, reg0_prediction = model_fit_valid(reg0)
print("Показатели для региона 1:")
reg1_valid, reg1_prediction = model_fit_valid(reg1)
print("Показатели для региона 2:")
reg2_valid, reg2_prediction = model_fit_valid(reg2)

Показатели для региона 0:
Cредний запас нефти: 92.59257
RMSE модели: 37.57942 

Показатели для региона 1:
Cредний запас нефти: 68.72855
RMSE модели: 0.89310 

Показатели для региона 2:
Cредний запас нефти: 94.96505
RMSE модели: 40.02971 



Проведя обучение модели и получив необходимые показатели, можно сделать следующие выводы:
Средние запасы нефти в регионах 0 и 2 выше чем в регионе 1, однако квадрат из средней квадратичной ошибки в обоих регионах намного выше, что означает слабую предсказуемость результатов для данных регионов. 

## Подготовка к расчёту прибыли

In [4]:
BUDGET = 10000000000
TOP_WELLS = 200
SAMPLE_WELLS = 500
WELL_INCOME = 450000
min_barrel = BUDGET / (TOP_WELLS * WELL_INCOME)
print('Минимальный запас нефти для одной скважины для безубыточной разработки: {:.2f}'.format(min_barrel))

Минимальный запас нефти для одной скважины для безубыточной разработки: 111.11


При условии отбора 200 лучших скважин при бюджете 10 млрд рублей на разработку, минимальным необходимым запасом является объем в 111.11 тыс. баррелей, что превышает средние показатели запасов нефти для всех регионов.

## Расчёт прибыли и рисков 

In [5]:
def income(predict, target):#строим функцию для расчета прибыли на вход прнимает пресказания модели и целевые значения
    predict = pd.Series(predict).reset_index(drop=True)#избавляемся от индексов
    target = pd.Series(target).reset_index(drop=True)#избавляемся от индексов
    predict_sorted = predict.sort_values(ascending=False)[:TOP_WELLS]# из предсказаний выделяем топ-200 вышек по запасам нефти
    fuel_quantity = target[predict_sorted.index].sum()#находим действительные запасы топлива в топ-200 вышек по предсказаниям
    operating_income = ((fuel_quantity*WELL_INCOME) - BUDGET) #считаем прибыль
    return operating_income

print('Предсказанная операционная прибыль для региона 0: {:.2f} рублей\n'.format(income(reg0_prediction, reg0_valid)))
print('Предсказанная операционная прибыль для региона 1: {:.2f} рублей\n'.format(income(reg1_prediction, reg1_valid)))
print('Предсказанная операционная прибыль для региона 2: {:.2f} рублей\n'.format(income(reg2_prediction, reg2_valid)))

Предсказанная операционной прибыли для региона 0: 3320826043.14 рублей

Предсказанная операционной прибыли для региона 1: 2415086696.68 рублей

Предсказанная операционной прибыли для региона 2: 2710349963.60 рублей



Если выбирать вышки на основании предсказаний нашей модели, то наиболее предпочтительным выглядит регион №0, со средним значением в 3.3. млрд рублей прибыли. Наименее предпочтительным выглядит регион №1 с его 2.4. млрд. рублей. Однако, поскольку предсказуемость результатов в регионах 0 и 2 оставляет желать лучшего, необходимо просчитать существующие риски и оценить возможность получения данных результатов случайно.

In [6]:
state = np.random.RandomState(12345)
def bootstrap(predict, target):# создаем функцию для перебора 1000 разных выборок размером в 500 вышек и расчета прибыли по топ-200 из них
    # функция принимает значения, предсказанные моделью и целевые значения
    target = target.reset_index(drop=True)#избавляемся от индексов
    values = []
    
    for i in range(1000):
        target_subsample = target.sample(n = SAMPLE_WELLS, replace=True, random_state=state)#выделяем подвыборку из 500 случайных строк
        predicted_subsample = predict[target_subsample.index]#находим соотвествующие предсказанные значения
        
        func = income(predicted_subsample, target_subsample)#применяем ранее созданную функцию income
        values.append(func)# присоединяем результаты к ранее созданному списку
    values = pd.Series(values)#переводи список в формат Series
    mean = values.mean()#находим среднее значение
        
    confidence_interval = [values.quantile(0.025), values.quantile(0.975)]# находим 95% доверительный интервал средней прибыли с помощью квантилей
    print("Средняя выручка:{:.2f}".format(mean))
    print("Доверительный интервал 95%:", confidence_interval)
    print("Риски убытка:{:.3f}".format((values < 0).mean()))
print('Средняя выручка и риски убытков для региона 0:')
bootstrap(reg0_prediction, reg0_valid)
print('\nСредняя выручка и риски убытков для региона 1:')
bootstrap(reg1_prediction, reg1_valid)
print('\nСредняя выручка и риски убытков для региона 2:')
bootstrap(reg2_prediction, reg2_valid)

Средняя выручка и риски убытков для региона 0:
Средняя выручка:396164984.80
Доверительный интервал: [-111215545.89049526, 909766941.5534225]
Риски убытка:0.069

Средняя выручка и риски убытков для региона 1:
Средняя выручка:461155817.28
Доверительный интервал: [78050810.7517417, 862952060.2637235]
Риски убытка:0.007

Средняя выручка и риски убытков для региона 2:
Средняя выручка:392950475.17
Доверительный интервал: [-112227625.37857565, 934562914.5511636]
Риски убытка:0.065


Наша проверка дала интересные результаты и предположение о случайности полученных средних значений оказалось верным. Низкая прогнозируемость для регионов 0 и 2 при формально бОльших запасах нефти может принести убытки при разработке. По итогу проверки можем рекомендовать для разрабоки регион 1.